## Preparación

In [2]:
##
## Apertura de la conexión
##
import sqlite3
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

In [8]:
##
## executescript() permite enviar varios comandos de SQL
## en la misma cadena de texto.
##

conn.executescript("""
DROP TABLE IF EXISTS truck_events;

CREATE TABLE truck_events (driverId       INT,
                           truckId        INT,
                           eventTime      STRING,
                           eventType      STRING,
                           longitude      DOUBLE,
                           latitude       DOUBLE,
                           eventKey       STRING,
                           correlationId  STRING,
                           driverName     STRING,
                           routeId        STRING,
                           routeName      STRING,
                           eventDate      STRING);



""")

conn.commit()

In [9]:
##
## La función execute() permite enviar únicamente un comando SQL
## al motor de la base de datos. El siguiente comando es equivalente
## a `SHOW TABLES;` en SQL
##
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [10]:
##
## La siguiente sentencia devuelve un string que
## representa los comandos sql para crear la tabla
##

cur.execute("SELECT sql FROM sqlite_master WHERE type='table' and name='truck_events'; ").fetchall()

[('CREATE TABLE truck_events (driverId       INT,\n                           truckId        INT,\n                           eventTime      STRING,\n                           eventType      STRING,\n                           longitude      DOUBLE,\n                           latitude       DOUBLE,\n                           eventKey       STRING,\n                           correlationId  STRING,\n                           driverName     STRING,\n                           routeId        STRING,\n                           routeName      STRING,\n                           eventDate      STRING)',)]

In [15]:
##
## Se imprime el comando sql equivalente para cada
## tabla existente en la base de datos
##
for a in cur.execute("SELECT sql FROM sqlite_master WHERE type='table';").fetchall():
    print(a[0])

CREATE TABLE truck_events (driverId       INT,
                           truckId        INT,
                           eventTime      STRING,
                           eventType      STRING,
                           longitude      DOUBLE,
                           latitude       DOUBLE,
                           eventKey       STRING,
                           correlationId  STRING,
                           driverName     STRING,
                           routeId        STRING,
                           routeName      STRING,
                           eventDate      STRING)


## Carga de datos usando INSERT INTO

In [ ]:
## Lectura de de todo el archivo

with open("truck_event_text_partition.csv", "rt") as f:
    data = f.readlines()

data = [line[:-1] if line[:-1] == "\n" else line for line in data]

#Separa ls campos por comas
data = [line.split(",") for line in data]

#Convierte la fina en una tupla
data = [tuple(line) for line in data]

#Descartar la cabecera
data = data[1:]

#Imprime los primeros 3 registros
data[0:3]

In [ ]:
## Cargar a partir de la list de tuplas contenidas en data

cur.executemany("INSERT INTO truck_events VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", data)

#Verificación

cur.execute("SELECT * FROM truck_events LIMIT 1;").fetchall

## Consulta de datos

In [ ]:
cur.execute("SELECT * FROM truck_events LIMIT 3; ").fetchall()

## Obtención de un subconjunto de registros

In [ ]:
conn.executescript("""
DROP TABLE IF EXISTS truc_events_subset;

CREATE TABLE truc_events_Subset
AS 
    SELECT * 
    FROM truck_events
    LIMIT 100

""")
conn.commit()

In [ ]:
cur.execute("SELECT * FROM truck_events_subset LIMIT1").fetchall()

## Obtención de un subconjunto de campos

In [ ]:
conn.executescript("""

DROP TABLE IF EXISTS specific_columns;

CREATE TABLE specific_columns
AS 
    SELECT
        driverID,
        eventTime,
        eventType,
    FROM
        truck_events_subset;

SELECT * FROM specific_columns LIMIT1;


""")

conn.commit()

In [ ]:
cur.execute("SELECT * FROM specific_columns LIMIT 2, 3;").fetchall()

In [ ]:
##
## Registros parael driverId==11

cur.execute("SELECT * FROM specific_columns WHERE driverID = 11;").fetchall()

In [ ]:
##
## Tipos de eventos
##
cur.execute("SELECT DISTINCT eventType FROM specific_columns;").fetchall()

### Escritura de la tabla en el disco

In [ ]:
## Obtención de los datos como una lista de tuplas

data = cur.execute("SELECT * FROM truck_events_subset;").fetchall()


## Conversión de los elementos de la tupla a strings

text = [[str(e) for e in row] for row in data]


## Concatenación de strings

text = [",".join(row) for row in text]
text = "\n".join(text)

#Escritura al disco

open("data.csv", "wt").write(text)

## Apéndice - Manejo de nulos

In [4]:
conn.executescript("""
DROP TABLE IF EXISTS nulltable;
CREATE TABLE nulltable(
    col1 INT,
    col2 VARCHAR(1)
);

INSERT INTO nulltable VALUES
    (1, "A"),
    (2, NULL),
    (NULL, "C"),
    (NULL, NULL),
    (5, "E");

""")
conn.commit()

In [6]:
cur.execute("SELECT * FROM nulltable;").fetchall()

[(1, 'A'), (2, None), (None, 'C'), (None, None), (5, 'E')]

In [7]:
cur.execute("SELECT * FROM nulltable WHERE col1 IS NULL;").fetchall()

[(None, 'C'), (None, None)]

In [8]:
cur.execute("SELECT * FROM nulltable WHERE col2 IS NULL;").fetchall()

[(2, None), (None, None)]

In [9]:
##
## condicionales
##

cur.execute("SELECT IFNULL(col1, 'Unknown'), IFNULL(col2, 'Unknown') FROM nulltable;").fetchall()

[(1, 'A'), (2, 'Unknown'), ('Unknown', 'C'), ('Unknown', 'Unknown'), (5, 'E')]